In [1]:
import requests
import defs
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instruments.pkl")

In [4]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [6]:
def get_candles_df(json_response):
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False: continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)
    

In [7]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pkl")

In [8]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [11]:
# p1 pair1
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")
print("Complete")

EUR_USD loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2021-01-28T22:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
USD_JPY loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
USD_CHF loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
USD_CAD loaded 3999 candles from 2021-01-28T23:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
GBP_USD loaded 3999 candles from 2021-01-28T22:00:00.000000000Z to 2021-09-20T12:00:00.000000000Z
GBP_JPY loaded 3999 